In [2]:
from discord_chatbot.module import food

None


In [3]:
menu = food.get_menu_data("푸름")
print(menu)

석식

흑미밥
두부김치국
안동찜닭
삼동초생채
옛맛깻잎지
백깍두기


In [ ]:
import re
from datetime import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions


def get_menu_data(user):
    result = []
    now = datetime.now()
    date = now.strftime("%m%d")
    time = now.strftime('%H')
    time = int(time)

    if user == '푸름':
        webpage = requests.get("https://dorm.kumoh.ac.kr/dorm/restaurant_menu01.do#")
        soup = BeautifulSoup(webpage.content, "html.parser")
        data = soup.find('table', {'class': 'smu-table tb-w150'})
        table = parser_functions.make2d(data)
        df = pd.DataFrame(data=table[1:], columns=table[0])
        # 각 칼럼 이름에서 숫자만 추출하여 새로운 칼럼 이름으로 설정
        new_columns = [re.sub("[^0-9]", "", column) for column in df.columns]
        df.columns = new_columns
        df.index = ['중식', '석식']
        for data in df[date]:
            result.append(data.replace('\n', '\n'))
        if 14 < time < 19:
            return result[1]
        else:
            return result[0]
        
    elif user == '오름1':
        webpage = requests.get("https://dorm.kumoh.ac.kr/dorm/restaurant_menu02.do#")
        soup = BeautifulSoup(webpage.content, "html.parser")
        data = soup.find('table', {'class': 'smu-table tb-w150'})
        table = parser_functions.make2d(data)
        df = pd.DataFrame(data=table[1:], columns=table[0])

        # 각 칼럼 이름에서 숫자만 추출하여 새로운 칼럼 이름으로 설정
        new_columns = [re.sub("[^0-9]", "", column) for column in df.columns]
        df.columns = new_columns

        df.index = ['중식', '석식']
        for data in df[date]:
            result.append(data.replace('\n', '\n'))
        if 14 < time < 19:
            return result[1]
        else:
            return result[0]
        
    elif user == '오름3':
        webpage = requests.get("https://dorm.kumoh.ac.kr/dorm/restaurant_menu03.do#")
        soup = BeautifulSoup(webpage.content, "html.parser")
        data = soup.find('table', {'class': 'smu-table tb-w150'})
        table = parser_functions.make2d(data)
        df = pd.DataFrame(data=table[1:], columns=table[0])
        # 각 칼럼 이름에서 숫자만 추출하여 새로운 칼럼 이름으로 설정
        new_columns = [re.sub("[^0-9]", "", column) for column in df.columns]
        df.columns = new_columns

        df.index = ['중식', '석식']
        for data in df[date]:
            result.append(data.replace('\n', '\n'))
        if 14 < time < 19:
            return result[1]
        else:
            return result[0]

    elif user == '학생식당':
        webpage = requests.get("https://www.kumoh.ac.kr/ko/restaurant01.do")
        soup = BeautifulSoup(webpage.content, "html.parser")

        data = soup.find('table', {'class': 'smu-table tb-w150'})

        table = parser_functions.make2d(data)

        df = pd.DataFrame(data=table[1:], columns=table[0])

        # 각 칼럼 이름에서 숫자만 추출하여 새로운 칼럼 이름으로 설정
        new_columns = [re.sub("[^0-9]", "", column) for column in df.columns]
        df.columns = new_columns

        df.index = ['조식', '중식']
        for data in df[date]:
            result.append(data.replace('\n', '\n'))
        if 11 < time < 14:
            return result[1]
        else:
            return result[0]


    elif user == '교직원식당':
        webpage = requests.get("https://www.kumoh.ac.kr/ko/restaurant02.do")

        soup = BeautifulSoup(webpage.content, "html.parser")

        data = soup.find('table', {'class': 'smu-table tb-w150'})

        table = parser_functions.make2d(data)

        df = pd.DataFrame(data=table[1:], columns=table[0])

        # 각 칼럼 이름에서 숫자만 추출하여 새로운 칼럼 이름으로 설정
        new_columns = [re.sub("[^0-9]", "", column) for column in df.columns]
        df.columns = new_columns

        df.index = ['중식', '석식']
        for data in df[date]:
            result.append(data.replace('\n', '\n'))
        if 14 < time < 19:
            return result[1]
        else:
            return result[0]


    elif user == '분식당':
        webpage = requests.get("https://www.kumoh.ac.kr/ko/restaurant04.do")

        soup = BeautifulSoup(webpage.content, "html.parser")

        data = soup.find('table', {'class': 'smu-table tb-w150'})

        table = parser_functions.make2d(data)

        df = pd.DataFrame(data=table[1:], columns=table[0])

        # 각 칼럼 이름에서 숫자만 추출하여 새로운 칼럼 이름으로 설정
        new_columns = [re.sub("[^0-9]", "", column) for column in df.columns]
        df.columns = new_columns

        df.index = ['일품요리']
        for data in df[date]:
            result.append(data.replace('\n', '\n'))

        return result[0]

    else:
        return '오류!'





In [26]:
from llama_index import SimpleDirectoryReader, download_loader

SimpleDirectoryReader = download_loader("SimpleDirectoryReader")



In [27]:
loader = SimpleDirectoryReader('./data',recursive=True, exclude_hidden = True)
documents = loader.load_data()

In [28]:
documents

[Document(id_='45da4e7d-757a-4e2f-8254-a9b85f2afda5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='5bb85fa19ee9f712858b03eb352907e77c1b4e8942ddd6e0eac7384eb41fca2b', text=': 0\n번호: 5835\n카테고리: [장학]\n제목: 2024년 삼성드림클래스 대학생멘토 선발 안내\n등록일: 2023-11-22\nlink: ?mode=view&articleNo=469669&article.offset=0&articleLimit=500', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='1875face-c1ac-4b73-8447-2846a96ff299', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='39c4835ca79159afa96824aae510e282683b3a3fdde2f2739cfa7bcb889944a9', text=': 1\n번호: 5834\n카테고리: [장학]\n제목: 2023년 2학기 동계방학「한국장학재단 취업연계유형 국가근로」참여학생 모집(~11/30까지)\n등록일: 2023-11-21\nlink: ?mode=view&articleNo=469627&article.offset=0&articleLimit=500', start_char_idx=None, end_char_idx=None, text_temp

In [13]:
import os
import openai
file_path = '/Users/dryoon04/Documents/GitHub/university-project/discord_chatbot/api key.txt'

# 파일 열기 (읽기 모드로 열기)
with open(file_path, 'r', encoding='utf-8') as file:
    # 파일 내용 읽어오기
    file_content = file.read()
os.environ["OPENAI_API_KEY"] = file_content
openai.api_key = os.getenv("OPENAI_API_KEY")

In [14]:
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper,ServiceContext
from langchain import OpenAI

In [15]:
llm_predictor = LLMPredictor(llm = OpenAI(temperature = 0, model_name = "text-embedding-ada-002	"))
max_input_size = 3900
num_output = 256
max_chunk_overlap = 1
PromptHelper = PromptHelper(max_input_size, num_output, max_chunk_overlap)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=PromptHelper)
index = GPTVectorStoreIndex.from_documents(documents, service_context = service_context)

In [21]:
index.storage_context.persist("notice_index")

from llama_index import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="notice_index")
new_index = load_index_from_storage(storage_context)
new_query_engine = new_index.as_query_engine()
response =  new_query_engine.query("대학원")
print(response)


The context information does not provide any specific information about the topic of "대학원" (graduate school). Therefore, it is not possible to provide a direct answer to the query based on the given context.


In [22]:
from llama_index import LLMPredictor, ServiceContext, VectorStoreIndex
from langchain.chat_models import ChatOpenAI

# Create a predictor using a custom model
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))

# Create a service context with the custom predictor
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# Create an index using the service context
custom_llm_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

custom_llm_query_engine = custom_llm_index.as_query_engine()
response = custom_llm_query_engine.query("대학원")
print(response)


AttributeError: module 'openai' has no attribute 'error'

In [32]:
from datetime import datetime
now = datetime.now()
date = now.strftime("%Y-%m-%d")
print(date)

2023-11-27


In [5]:
from module import chatgpt as gpt


print(gpt.chat("가장최근의 공지사항을 알려줘"))



APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [6]:
! pip list

Package                                  Version
---------------------------------------- ------------
aiofiles                                 22.1.0
aiohttp                                  3.9.0
aiosignal                                1.3.1
aiosqlite                                0.18.0
aiostream                                0.5.2
annotated-types                          0.6.0
anyio                                    3.7.1
appnope                                  0.1.3
argon2-cffi                              21.3.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asgiref                                  3.7.2
asttokens                                2.2.1
async-timeout                            4.0.2
attrs                                    23.1.0
Babel                                    2.11.0
backcall                                 0.2.0
backoff                                  2.2.1
backports.functools-lru-cache            1.6.